<a href="https://colab.research.google.com/github/BF667/Nooshell-RVC/blob/main/Nooshell_RVC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nooshell RVC



___


* this colab made by [BF667](https://github.com/BF667)

In [ ]:
from IPython.display import clear_output
from google.colab import drive
import codecs



print("Installing...")
#@title # **Installation**
#@markdown Check the box below to mount Google Drive
use_drive = False  #@param {type:"boolean"}
if use_drive:
  drive.mount('/content/drive', force_remount=True)

url = "https://github.com/BF667/Nooshell-RVC.git"


pipr = "Nooshell-RVC/requirements.txt"








!git clone $url > /dev/null 2>&1
!pip install -r $pipr > /dev/null 2>&1
!apt-get update > /dev/null 2>&1
!apt-get install -y libcudnn8 > /dev/null 2>&1


clear_output()
print('Installation done!')

In [ ]:
#@title Run UI
import codecs
import os
path = "/content/Nooshell-RVC"
%cd $path
app = codecs.decode('ncc.cl','rot_13')

#@markdown The type of tunnel you wanna use for seeing the public link, so that if one of them is down, you can use the other one.
Tunnel = "Gradio" #@param ["Gradio", "Ngrok", "Cloudflare", "LocalTunnel", "Horizon"]

#@markdown Also when using Ngrok, Cloudflare, LocalTunnel or Horizon, you need to wait for the Local URL to appear, and only after that click on the Public URL which is above.

#@markdown Use the following option **only if you chose Ngrok** as the Tunnel:

#@markdown You can get the Ngrok Tunnel Authtoken here: https://dashboard.ngrok.com/tunnels/authtokens/new.
ngrok_authtoken = "" #@param {type:"string"}

# @markdown You can optinally change the Ngrok Tunnel Region to one nearer to you for lower latency
ngrok_region = "us - United States (Ohio)" # @param ["au - Australia (Sydney)","eu - Europe (Frankfurt)", "ap - Asia/Pacific (Singapore)", "us - United States (Ohio)", "jp - Japan (Tokyo)", "in - India (Mumbai)","sa - South America (Sao Paulo)"]


#@markdown Use the following option **only if you chose Horizon** as the Tunnel:

#@markdown You can get the Horizon ID here: https://hrzn.run/dashboard/ , login, on the 2nd step, it shows an `hrzn login YOUR_ID`, you need to copy that id.
horizon_id = "" #@param {type:"string"}

if Tunnel =="Gradio":
  share_flag = "--share"
elif Tunnel == "Ngrok":
  share_flag = ""
  from pyngrok import conf, ngrok
  NgrokConfig = conf.PyngrokConfig()
  NgrokConfig.auth_token = ngrok_authtoken
  NgrokConfig.region = ngrok_region[0:2]
  conf.set_default(NgrokConfig)
  http_tunnel = ngrok.connect(4000, bind_tls=True)
  print("Ngrok Tunnel Public URL:", http_tunnel.public_url)
elif Tunnel == "Cloudflare":
  share_flag = ""
  # download cloudflare
  !curl -LO https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
  !dpkg -i cloudflared-linux-amd64.deb
  !rm -rf nohup.out
  import time
  # Run cloudflare
  !nohup cloudflared tunnel --url localhost:4000 &
  clear_output()
  time.sleep(5)
  # Find and print the Cloudflare URL with a prefix
  cloudflare_url = !grep -oE "https://[a-zA-Z0-9.-]+\.trycloudflare\.com" nohup.out
  print(f"Cloudflare Tunnel Public URL: {cloudflare_url[0]}")
elif Tunnel == "LocalTunnel":
  share_flag = ""
  # install
  !npm install -g localtunnel
  import time
  import urllib
  # run localtunnel
  with open('url.txt', 'w') as file:
        file.write('')

  get_ipython().system_raw('lt --port 4000 >> url.txt 2>&1 &')

  time.sleep(4)

  endpoint_ip = urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n")

  with open('url.txt', 'r') as file:
      tunnel_url = file.read()
      tunnel_url = tunnel_url.replace("your url is: ", "")

  clear_output()
  print(f"LocalTunnel Tunnel Public URL: \033[0m\033[93m{tunnel_url}\033[0m", end="\033[0m")

  print(f'LocalTunnel Password: {endpoint_ip}')
elif Tunnel == "Horizon":
  share_flag = ""
  # install
  !npm install -g @hrzn/cli
  import time
  # login
  !hrzn login $horizon_id
  # run horizon
  with open('url.txt', 'w') as file:
        file.write('')

  get_ipython().system_raw('hrzn tunnel http://localhost:4000 >> url.txt 2>&1 &')

  time.sleep(4)

  with open('url.txt', 'r') as file:
      tunnel_url = file.read()
      tunnel_url = !grep -oE "https://[a-zA-Z0-9.-]+\.hrzn\.run" url.txt
      tunnel_url = tunnel_url[0]

  clear_output()

  print(f"Horizon Tunnel Public URL: \033[0m\033[93m{tunnel_url}\033[0m")

# kills previously running processes
!fuser -k 9999/tcp

command = f"python {app} {share_flag}"
!{command}

### Thanks to People Who Contributed to the project:
<a href="https://github.com/Politrees-Kanoyo/Nooshell-RVC/graphs/contributors" target="_blank">
  <img src="https://contrib.rocks/image?repo=Politrees-Kanoyo/Nooshell-RVC" />
</a>